# NoteBook To Check Point or Polygon Inside Polygon

The objective of this notebook is to search a set of points or polygons for spatial overlap with a largerset

By Cascsde Tuholske 2019-01-17

#### Notes
1. Points should be dialated to circles to prevent one falling just outside the larger polygon set (see Lusaka)

### Depedencies

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import Polygon
from shapely.geometry import shape
import ast
from shapely.geometry import mapping
import rasterio
import copy

### Functions

In [2]:
def load_points (file):
    """ This function loads a csv 
    of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [3]:
def point_buffer(gpd_df, raduis):
    "Function to make a shapely polygon buffer around a point"
   
    # AGU 2018-12-04 - radius set to ~250m at the equator 
    #radius = 250*1/(111*1000)

    # new_gpd_df = gpd.GeoDataFrame()
    gpd_df.rename(columns={'geometry':'old_geom'}, inplace=True)
    
    arr = []
    
    for point in gpd_df['old_geom']:
        buffer = point.buffer(radius)
        arr.append((buffer))
    
    gpd_df['geometry'] = arr
    
    return gpd_df

In [4]:
def poly_point (poly, point):
    """
    This function will check if points are inside polygons if given two gpd dataframes with points and polygons
    Returns point ids, point geometry, polygon index # and polygon geometry
    """
    
    out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
    for index_point, row_point in point.iterrows():
        for index_poly, row_poly in poly.iterrows():
            if row_point['geometry'].within(row_poly['geometry']):
                point_id = row_point['id']
                point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
                poly_id = index_poly
                poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
                out_arr.append((point_id, 
                                point_geom, 
                                poly_id, 
                                poly_geom))

    return out_arr

# Note 2018-11-30 update arr to gpd_df 

In [24]:
def poly_overlap (point_buffer, poly_raster):
    """
    This function will check if point buffers intersect with polygons 
    if given two gpd dataframes with point buffers and polygons
    Returns point ids, point geometry, polygon index #, and polygon geometry in a geopandas DF.
    It goes faster if smaller list goes first
    """
    
    # make arrays to fill
    country_arr = []
    city_arr = []
    lat_arr = []
    lon_arr = []
    point_geom_arr = []
    osm_id_arr = [] 
    FID_arr = [] 
    #poly_geom_arr = []
    
    for index_point_buffer, row_point_buffer in point_buffer.iterrows():
        for index_poly_raster, row_poly_raster in poly_raster.iterrows():
            if row_point_buffer['geometry'].intersects(row_poly_raster['geometry']):
                country = row_point_buffer['country']
                city = row_point_buffer['city']
                lat = row_point_buffer['lat']
                lon = row_point_buffer['lon']
                point_geom = row_point_buffer['old_geom']
                osm_id = row_point_buffer['osm_id']
                poly_id = row_poly_raster['FID']
                poly_geom = shape(mapping(row_poly_raster['geometry'])) # make polygon 

                country_arr.append((country))
                city_arr.append((city))
                lat_arr.append((lat))
                lon_arr.append((lon))
                point_geom_arr.append((point_geom))
                osm_id_arr.append((osm_id))
                FID_arr.append((poly_id))
                #poly_geom_arr.append((poly_geom))
    
    # put results into a geopandas df
    new_gpd_df = gpd.GeoDataFrame()
    new_gpd_df['osm_id'] = osm_id_arr
    new_gpd_df['FID'] = FID_arr
    new_gpd_df['geometry'] = poly_geom_arr
    new_gpd_df['country'] = country_arr           
    new_gpd_df['city'] = city_arr
    new_gpd_df['lat'] = lat_arr
    new_gpd_df['lon'] = lon_arr
    #new_gpd_df['point_geom'] = point_geom_arr
    
    
    return new_gpd_df

# Note 2018-11-30 update arr to gpd_df 

### Analysis

In [10]:
# File Paths

# data = ''
temp_data = '../../temp_data/'
osm_point_file = '20190114_osm_africa_cities.csv'
poly_raster_file = 'AFR_PPP_2015_adj_v2_1500c300.shp'

In [11]:
# Load OSM Points
osm_point_gdf = load_points(temp_data+osm_point_file)
print(osm_point_gdf.head(6))
print(len(osm_point_gdf))

   country     osm_id         city        lat       lon  \
0  Algeria   27564946  Constantine  36.364519  6.608260   
1  Algeria   27565103         Oran  35.703275 -0.649298   
2  Algeria  253292160   Tizi Ouzou  36.713784  4.049392   
3  Algeria  262963217      El Oued  33.361177  6.860349   
4  Algeria  262964638    Touggourt  33.109897  6.066102   
5  Algeria  264573224     Laghouat  33.806352  2.880862   

                                 geometry  
0               POINT (6.60826 36.364519)  
1  POINT (-0.6492975999999999 35.7032751)  
2            POINT (4.0493919 36.7137843)  
3            POINT (6.8603492 33.3611766)  
4             POINT (6.066102 33.1098968)  
5            POINT (2.8808616 33.8063518)  
876


In [13]:
# Add column for type: city or town
osm_point_gdf['type'] = 'city' 
osm_point_gdf.head(6)

,country,osm_id,city,lat,lon,geometry,type
0,Algeria,27564946,Constantine,36.364519,6.608260,POINT (6.60826 36.364519),city
1,Algeria,27565103,Oran,35.703275,-0.649298,POINT (-0.6492975999999999 35.7032751),city
2,Algeria,253292160,Tizi Ouzou,36.713784,4.049392,POINT (4.0493919 36.7137843),city
3,Algeria,262963217,El Oued,33.361177,6.860349,POINT (6.8603492 33.3611766),city
4,Algeria,262964638,Touggourt,33.109897,6.066102,POINT (6.066102 33.1098968),city
5,Algeria,264573224,Laghouat,33.806352,2.880862,POINT (2.8808616 33.8063518),city


In [15]:
# AGU 2018-12-04 - radius set to ~250m at the equator 

radius = 250*1/(111*1000)
radius

0.0022522522522522522

In [16]:
osm_buffer_gdf = point_buffer(osm_point_gdf, radius)

In [17]:
print(len(osm_buffer_gdf))

876
   country     osm_id         city        lat       lon  \
0  Algeria   27564946  Constantine  36.364519  6.608260   
1  Algeria   27565103         Oran  35.703275 -0.649298   
2  Algeria  253292160   Tizi Ouzou  36.713784  4.049392   
3  Algeria  262963217      El Oued  33.361177  6.860349   
4  Algeria  262964638    Touggourt  33.109897  6.066102   
5  Algeria  264573224     Laghouat  33.806352  2.880862   

                                 old_geom  type  \
0               POINT (6.60826 36.364519)  city   
1  POINT (-0.6492975999999999 35.7032751)  city   
2            POINT (4.0493919 36.7137843)  city   
3            POINT (6.8603492 33.3611766)  city   
4             POINT (6.066102 33.1098968)  city   
5            POINT (2.8808616 33.8063518)  city   

                                            geometry  
0  POLYGON ((6.610512252252253 36.364519, 6.61050...  
1  POLYGON ((-0.6470453477477477 35.7032751, -0.6...  
2  POLYGON ((4.051644152252252 36.7137843, 4.0516...  
3  P

In [ ]:
poly_overlap_out.head(6)

In [19]:
# Load Polygon

raster_polygons = gpd.read_file(temp_data+poly_raster_file)
print(len(raster_polygons))

12429


In [ ]:
# run poly_overlap

import time
checkpoint = time.time()

poly_overlap_out = poly_overlap(osm_buffer_gdf, raster_polygons)

print("elapsed time is: {}s".format(time.time()-checkpoint))

In [ ]:
print(len(poly_overlap_out))

In [ ]:
poly_overlap_out.head(6)

In [ ]:
poly_overlap_out.to_file(temp_data+'.shp', driver='ESRI Shapefile')